In [1]:
import pandas as pd
import numpy as np
#import matplotlib
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

import dropbox
from zipfile import ZipFile
import pandas as pd
from utils.data_extract_utils import extract_zip

# Connect to dropbox
dbx = dropbox.Dropbox('cHV7yAR0J6YAAAAAAAAAAVQ1NLCrOwerbaNltPWHslYXKuUTJ5_wfgJsuFcmx83o')

In [2]:
# Download, and extract data from dropbox into memory. 
data = {}
for entry in dbx.files_list_folder('').entries:
    response = dbx.files_download('/{}'.format(entry.name))
    
    if 'zip' in entry.name:
        content = extract_zip(response[1].content)
    
        for file in content:
            df = pd.read_csv(file[1])
            data[entry.name.replace('.csv.zip', '')] = df
            

**note**

- make the code above into a function and add it to the utils module 
- also add a carrage return i.e a loading bar 

# Create a complete DataFrame
- a dataframe that contains all of the features 

In [4]:
data.keys()

dict_keys(['application_test', 'application_train', 'bureau_balance', 'bureau', 'credit_card_balance', 'installments_payments', 'POS_CASH_balance', 'previous_application'])

In [5]:
## create tables 
application_train = data['application_train']
bureau_balance = data['bureau_balance']
bureau = data['bureau']
credit_card_balance = data['credit_card_balance']
installments_payments = data['installments_payments']
POS_CASH_balance = data['POS_CASH_balance']
previous_application = data['previous_application']
application_test = data['application_test']

In [10]:
def OHE(df, contain_nan=True):
    all_cols = list(df.columns)
    cat_col = list(df.select_dtypes(include=object).columns)
    df = pd.get_dummies(df, columns=cat_col, dummy_na=contain_nan)
    new_cols = set(list(df.columns)).difference(set(all_cols))
    return new_cols, df

In [11]:
cc, _ = OHE(application_train, contain_nan=True)

In [14]:
len(cc)

156

In [6]:
application_train.select_dtypes(include=object).columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')

In [16]:
## combine train and test set
train_test = application_train.append(application_test).reset_index()

In [17]:
train_test.shape

(356255, 123)

In [18]:
application_train.shape

(307511, 122)

In [19]:
application_test.shape

(48744, 121)

In [20]:
307511+48744

356255

In [25]:
from data_extract_utils import extract_features_from_bureau

In [23]:
bb = extract_features_from_bureau(bureau, bureau_balance)

NameError: name 'data' is not defined

In [26]:
bb = extract_features_from_bureau(bureau, bureau_balance, data)

TypeError: extract_features_from_bureau() takes 2 positional arguments but 3 were given

In [27]:
## Amerius functions 
def extract_features_from_bureau(bureau_df, bureau_balances_df):
    bureau_df['AMT_CREDIT_SUM_DEBT'] = bureau_df['AMT_CREDIT_SUM_DEBT'].fillna(value=0)
    bureau_df['AMT_CREDIT_DEBT_RATIO'] =  bureau_df['AMT_CREDIT_SUM_DEBT']/data['bureau']['AMT_CREDIT_SUM']
    bureau_df['AMT_CREDIT_DEBT_RATIO'] = bureau_df['AMT_CREDIT_DEBT_RATIO'].fillna(value=0)
    
    
    bureau_df = bureau_df.set_index('SK_ID_BUREAU')
    
    DPD_STATUS_MAP = {
        'C': 0, 
        'X': 0,
        '0': 0,
        '1': 1,
        '2': 2,
        '3': 3,
        '4': 4,
        '5': 5,
    }
    def sum_of_dpd(x):
        return x['STATUS'].map(DPD_STATUS_MAP).sum()

    dpd_counts_df = bureau_balances_df.groupby(['SK_ID_BUREAU']).apply(sum_of_dpd)
    
    dpd_counts_df = pd.DataFrame(dpd_counts_df, columns=['DPD_COUNTS']);
    bureau_with_dpds = pd.concat([bureau_df, dpd_counts_df], axis=1)
    
    bureau_with_dpds = bureau_with_dpds.set_index('SK_ID_CURR')
    
    return bureau_with_dpds[:][['AMT_CREDIT_DEBT_RATIO', 'CREDIT_DAY_OVERDUE', 'DPD_COUNTS']]

In [28]:
bb = extract_features_from_bureau(bureau, bureau_balance)

In [31]:
bb.shape

(1759469, 3)

In [32]:
bb.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1759469 entries, 166497.0 to 134450.0
Data columns (total 3 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   AMT_CREDIT_DEBT_RATIO  float64
 1   CREDIT_DAY_OVERDUE     float64
 2   DPD_COUNTS             float64
dtypes: float64(3)
memory usage: 53.7 MB


In [33]:
bb['DPD_COUNTS'].isnull().sum()

942074

In [34]:
942074/1759469

0.5354308601060889

In [30]:
bb.head(20)

,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS
SK_ID_CURR,,,
166497.0,0.000000,0.0,NaN
166497.0,0.000000,0.0,NaN
166497.0,0.000000,0.0,NaN
166497.0,0.000000,0.0,NaN
166497.0,1.000000,0.0,NaN
356385.0,0.312983,0.0,NaN
375916.0,0.000000,0.0,NaN
375916.0,0.000000,0.0,NaN
375916.0,0.000000,0.0,NaN


In [36]:
bb.reset_index(inplace=True)
bb.head()

,SK_ID_CURR,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS
0,166497.0,0.0,0.0,NaN
1,166497.0,0.0,0.0,NaN
2,166497.0,0.0,0.0,NaN
3,166497.0,0.0,0.0,NaN
4,166497.0,1.0,0.0,NaN


In [42]:
bb['SK_ID_CURR'] = bb['SK_ID_CURR'].astype('Int64')
bb.head()

,SK_ID_CURR,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS
0,166497,0.0,0.0,NaN
1,166497,0.0,0.0,NaN
2,166497,0.0,0.0,NaN
3,166497,0.0,0.0,NaN
4,166497,1.0,0.0,NaN


In [44]:
train_test['SK_ID_CURR'] = train_test['SK_ID_CURR'].astype('Int64')

In [47]:
bb['SK_ID_CURR'].dtype

Int64Dtype()

In [48]:
train_test['SK_ID_CURR'].dtype

Int64Dtype()

In [50]:
train_test_bb = train_test.merge(bb, how='left', on='SK_ID_CURR')

In [51]:
train_test_bb.head()

,index,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS
0,0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,6.0
1,0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.54618,0.0,6.0
2,0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,1.0
3,0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,0.0
4,0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,6.0


In [52]:
train_test_bb.shape

(1766872, 126)

In [54]:
bureau.shape

(1716428, 18)

In [60]:
df = train_test.join(bb, how='left', on='SK_ID_CURR', lsuffix='_left', rsuffix='_right',)

In [61]:
df.shape

(356255, 127)

In [62]:
df.head()

,index,SK_ID_CURR_left,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SK_ID_CURR_right,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS
0,0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,...,0.0,0.0,0.0,0.0,0.0,1.0,267358,0.00000,0.0,1.0
1,1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,...,0.0,0.0,0.0,0.0,0.0,0.0,<NA>,NaN,NaN,1.0
2,2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,267358,0.00000,0.0,2.0
3,3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,...,NaN,NaN,NaN,NaN,NaN,NaN,186604,0.00000,0.0,0.0
4,4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,186604,0.99725,0.0,0.0


In [ ]:
## something is fishhy 

In [57]:
DPD_STATUS_MAP = {
    'C': 0, 
    'X': 0,
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
}
def sum_of_dpd(x):
    return x['STATUS'].map(DPD_STATUS_MAP).sum()

dpd_counts_df = bureau_balance.groupby(['SK_ID_BUREAU']).apply(sum_of_dpd)

In [58]:
dpd_counts_df.shape

(817395,)

In [59]:
dpd_counts_df

SK_ID_BUREAU
5001709     0
5001710     0
5001711     0
5001712     0
5001713     0
           ..
6842884     0
6842885    60
6842886     0
6842887     0
6842888     1
Length: 817395, dtype: int64

In [65]:
## bingo
bb.SK_ID_CURR.is_unique

False

In [66]:
bb.head()

,SK_ID_CURR,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS
0,166497,0.0,0.0,NaN
1,166497,0.0,0.0,NaN
2,166497,0.0,0.0,NaN
3,166497,0.0,0.0,NaN
4,166497,1.0,0.0,NaN


In [68]:
new_bb = bb.groupby('SK_ID_CURR').agg({'AMT_CREDIT_DEBT_RATIO': 
                                       'mean', 'CREDIT_DAY_OVERDUE': 'mean',
                                      'DPD_COUNTS': 'mean'})

In [69]:
new_bb.head()

,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS
SK_ID_CURR,,,
100001,0.282518,0.0,0.142857
100002,0.068272,0.0,3.375000
100003,0.000000,0.0,NaN
100004,0.000000,0.0,NaN
100005,0.601256,0.0,0.000000


In [70]:
new_bb.shape

(305811, 3)

In [71]:
train_test.shape

(356255, 123)

In [73]:
305811/356255

0.8584047943186762

In [74]:
df = train_test.join(new_bb, how='left', on='SK_ID_CURR', lsuffix='_left', rsuffix='_right')

In [75]:
df.head()

,index,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS
0,0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.068272,0.0,3.375
1,1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN
2,2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN
3,3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN


In [76]:
df.shape

(356255, 126)

In [55]:
bureau_balance.shape

(27299925, 3)

In [53]:
356255 > 1766872

False

In [35]:
bb[bb['CREDIT_DAY_OVERDUE'] > 0]

,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS
SK_ID_CURR,,,
176066.0,0.518866,20.0,NaN
287908.0,0.740034,319.0,NaN
449409.0,0.000000,2187.0,45.0
226496.0,0.000000,8.0,2.0
402077.0,0.726311,22.0,3.0
...,...,...,...
339321.0,0.708440,14.0,NaN
221708.0,0.012750,2654.0,NaN
212415.0,0.901193,8.0,0.0


In [15]:
application_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis = 1)

0         0.161787
1         0.466757
2         0.642739
3         0.650442
4         0.322738
            ...   
307506    0.413601
307507    0.115992
307508    0.499536
307509    0.587593
307510    0.518984
Length: 307511, dtype: float64

In [2]:
application_train = pd.read_csv('home-credit-default-risk/application_train.csv')
application_test = pd.read_csv('home-credit-default-risk/application_test.csv')

In [3]:
from sklearn.model_selection import train_test_split

In [5]:
X, y = application_train.drop('TARGET', axis=1), application_train['TARGET']
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  test_size=0.2, 
                                                  random_state=42,
                                                  stratify = y) 
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(246008, 121) (61503, 121) (246008,) (61503,)


In [8]:
y_train.value_counts()

0    226148
1     19860
Name: TARGET, dtype: int64

In [9]:
226148/(226148+19860)

0.9192709180189262

In [14]:
categorical_features = list(application_train.select_dtypes(include=object))
numerical_features = list(application_train.select_dtypes(include=float))
discreate_features = list(application_train.select_dtypes(include=int))

In [11]:
categorical_features

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [12]:
numerical_features

['AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_REGISTRATION',
 'OWN_CAR_AGE',
 'CNT_FAM_MEMBERS',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_M

In [15]:
discreate_features

['SK_ID_CURR',
 'TARGET',
 'CNT_CHILDREN',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_ID_PUBLISH',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [16]:
application_train.shape

(307511, 122)

In [18]:
NM = application_train.dropna()

In [19]:
NM['TARGET'].value_counts()

0    8076
1     526
Name: TARGET, dtype: int64

In [20]:
526/(8076+526)

0.06114857009997675

In [21]:
application_test.isna().sum()

SK_ID_CURR                       0
NAME_CONTRACT_TYPE               0
CODE_GENDER                      0
FLAG_OWN_CAR                     0
FLAG_OWN_REALTY                  0
                              ... 
AMT_REQ_CREDIT_BUREAU_DAY     6049
AMT_REQ_CREDIT_BUREAU_WEEK    6049
AMT_REQ_CREDIT_BUREAU_MON     6049
AMT_REQ_CREDIT_BUREAU_QRT     6049
AMT_REQ_CREDIT_BUREAU_YEAR    6049
Length: 121, dtype: int64

In [22]:
application_test.shape

(48744, 121)

In [23]:
ohe_train = pd.get_dummies(application_train)
ohe_test = pd.get_dummies(application_test)

print('train shape:', ohe_train.shape) 
print('test shape:', ohe_test.shape) 

train shape: (307511, 246)
test shape: (48744, 242)


In [24]:
ohe_train_cols = set(ohe_train.columns)
ohe_test_cols = set(ohe_test.columns)

print(ohe_train_cols.difference(ohe_test_cols))

{'NAME_INCOME_TYPE_Maternity leave', 'CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Unknown', 'TARGET'}


In [25]:
y = ohe_train['TARGET']
## align dataframes 
X, X_test = ohe_train.align(ohe_test, join='inner', axis=1)
print('train shape:', X.shape)
print('test shape:', X_test.shape)

train shape: (307511, 242)
test shape: (48744, 242)


In [26]:
# basic preproccessing based on eda part 1

## DAYS_EMPLOYED: train
X['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
X['DAYS_EMPLOYED'] =X['DAYS_EMPLOYED'].apply(lambda x: abs(x))

X['BIRTH_IN_YEARS'] = X['DAYS_BIRTH'].apply(lambda x: abs(x)/365)

# DAYS_EMPLOYED: test
X_test['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
X_test['DAYS_EMPLOYED'] =X_test['DAYS_EMPLOYED'].apply(lambda x: abs(x))

X_test['BIRTH_IN_YEARS'] = X_test['DAYS_BIRTH'].apply(lambda x: abs(x)/365)


In [28]:
## simple - drop all na isntances 
X_simple = X.dropna()
X_test_simple = X_test.dropna()
print('train shape:', X_simple.shape)
print('test shape:', X_test_simple.shape)

train shape: (10746, 243)
test shape: (2221, 243)


In [29]:
X_simple.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,BIRTH_IN_YEARS
71,100083,0,103500.0,573628.5,24435.0,463500.0,0.009657,-15406,892.0,-341.0,...,0,0,0,0,0,1,0,1,0,42.208219
124,100145,1,202500.0,260725.5,16789.5,198000.0,0.018850,-16282,4375.0,-762.0,...,0,0,0,0,1,0,0,1,0,44.608219
152,100179,0,202500.0,675000.0,53329.5,675000.0,0.031329,-11375,2311.0,-180.0,...,0,0,1,0,0,0,0,1,0,31.164384
161,100190,0,162000.0,263686.5,24781.5,238500.0,0.022625,-13972,4472.0,-464.0,...,0,0,0,0,1,0,0,1,0,38.279452
164,100193,0,225000.0,296280.0,15124.5,225000.0,0.020246,-17879,940.0,-2294.0,...,0,0,0,0,0,1,0,1,0,48.983562


In [35]:
y_simple = y[list(X_simple.index)]
y_simple.shape

(10746,)

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

In [38]:
Test_simple_ids = X_test_simple ['SK_ID_CURR']
X_simple.drop(columns= ['SK_ID_CURR'], inplace=True)
X_test_simple.drop(columns= ['SK_ID_CURR'], inplace=True)


/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [42]:
X_train, X_val, y_train, y_val = train_test_split(X_simple, 
                                                  y_simple,
                                                  test_size=0.2,
                                                  random_state=42,
                                                  stratify=y_simple)

In [43]:
y_train.value_counts()

0    8070
1     526
Name: TARGET, dtype: int64

In [44]:
8070/(8070+ 526)

0.9388087482550024

In [45]:
y_simple.value_counts()

0    10089
1      657
Name: TARGET, dtype: int64

In [46]:
10089/(10089+657)

0.9388609715242882

In [54]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)

In [55]:
cross_val_score(dummy_clf, X_train, y_train, cv =5, scoring='accuracy').mean()

0.9388087990584033

In [56]:
dummy_clf.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

In [57]:
y1_test = dummy_clf.predict(X_test)

In [58]:
dummy_clf.score(X_val, y_val)

0.9390697674418604

In [60]:
rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [61]:
rfc.feature_importances_

array([4.29253057e-03, 1.46068888e-02, 1.51585195e-02, 1.57614673e-02,
       1.51747765e-02, 1.30419945e-02, 1.90878442e-02, 1.96878342e-02,
       2.32358546e-02, 1.84647631e-02, 1.59127107e-02, 0.00000000e+00,
       0.00000000e+00, 2.74324971e-03, 2.37338350e-04, 2.92610002e-03,
       2.24787590e-03, 5.50541029e-03, 3.42297223e-03, 3.37859418e-03,
       1.19063678e-02, 5.55431361e-04, 1.18557122e-03, 1.60975209e-03,
       9.20389622e-04, 2.20454886e-03, 1.95245992e-03, 3.84460781e-02,
       3.41291831e-02, 4.07905509e-02, 1.30004859e-02, 1.26715863e-02,
       1.20167038e-02, 1.27317273e-02, 1.53877721e-02, 4.99231176e-03,
       8.62220866e-03, 5.30670902e-03, 6.23862267e-03, 1.47794364e-02,
       1.28847339e-02, 1.54140830e-02, 5.81738860e-03, 8.03348271e-03,
       1.17484160e-02, 1.35738990e-02, 1.24400408e-02, 1.21256741e-02,
       1.37341653e-02, 4.20967065e-03, 6.90681114e-03, 5.18947235e-03,
       6.98124620e-03, 1.47820531e-02, 1.18881889e-02, 1.44134801e-02,
      

In [63]:
features = list(X_simple.columns)

In [64]:
fm = {'feature': features,
    'importance': rfc.feature_importances_}
feature_importance = pd.DataFrame(fm)
feature_importance.sort_values(by='importance', ascending=False).head(10)

,feature,importance
29,EXT_SOURCE_3,0.040791
27,EXT_SOURCE_1,0.038446
28,EXT_SOURCE_2,0.034129
8,DAYS_REGISTRATION,0.023236
7,DAYS_EMPLOYED,0.019688
6,DAYS_BIRTH,0.019088
9,DAYS_ID_PUBLISH,0.018465
241,BIRTH_IN_YEARS,0.018245
10,OWN_CAR_AGE,0.015913
3,AMT_ANNUITY,0.015761


In [65]:
rfc.predict_proba(X_train)

array([[0.97, 0.03],
       [0.99, 0.01],
       [0.99, 0.01],
       ...,
       [0.97, 0.03],
       [0.99, 0.01],
       [0.98, 0.02]])

In order to submit to kaggle - we can't drop any instances from the test set i.e the submission file needs to contain 48744 instances. 

In [69]:
# change our approach
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='median')

Test_ids = X_test['SK_ID_CURR']
X.drop(columns= ['SK_ID_CURR'], inplace=True)
X_test.drop(columns= ['SK_ID_CURR'], inplace=True)

imp.fit(X)
X1 = imp.transform(X)
X1_test = imp.transform(X_test)

In [70]:
X_train, X_val, y_train, y_val = train_test_split(X1, 
                                                  y,
                                                  test_size=0.2,
                                                  random_state=42,
                                                  stratify=y)

In [71]:
y_train.value_counts()

0    226148
1     19860
Name: TARGET, dtype: int64

In [72]:
226148/(226148+19860)

0.9192709180189262

In [73]:
y.value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [74]:
282686/(282686+24825)

0.9192711805431351

In [75]:
rfc1 = RandomForestClassifier(random_state=42, n_jobs=-1)
rfc1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [78]:
features = list(X.columns)
fm = {'feature': features,
    'importance': rfc.feature_importances_}
feature_importance = pd.DataFrame(fm)
feature_importance.sort_values(by='importance', ascending=False).head(10)

,feature,importance
29,EXT_SOURCE_3,0.040791
27,EXT_SOURCE_1,0.038446
28,EXT_SOURCE_2,0.034129
8,DAYS_REGISTRATION,0.023236
7,DAYS_EMPLOYED,0.019688
6,DAYS_BIRTH,0.019088
9,DAYS_ID_PUBLISH,0.018465
241,BIRTH_IN_YEARS,0.018245
10,OWN_CAR_AGE,0.015913
3,AMT_ANNUITY,0.015761


In [79]:
rfc1.score(X_val, y_val)

0.9193535274702047

In [82]:
rfc_test_pred = rfc.predict_proba(X1_test)[:,1]

In [83]:
submission1_dict = {'SK_ID_CURR': Test_ids, 
            'TARGET': rfc_test_pred}
rfc_submission1 = pd.DataFrame(submission1_dict)
rfc_submission1.head()

,SK_ID_CURR,TARGET
0,100001,0.11
1,100005,0.03
2,100013,0.06
3,100028,0.03
4,100038,0.04


In [84]:
rfc_submission1.to_csv('submission1.csv', index=False)

In [85]:
logreg = LogisticRegression(solver='lbfgs')

In [87]:
logreg.fit(X_train, y_train)

/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [88]:
logreg.score(X_train, y_train)

0.9192627882020097

In [89]:
logreg.score(X_val, y_val)

0.9192722306228964

In [90]:
logreg_test_pred = logreg.predict_proba(X1_test)[:,1]

In [91]:
submission2_dict = {'SK_ID_CURR': Test_ids, 
            'TARGET': logreg_test_pred }
logreg_submission2 = pd.DataFrame(submission2_dict)
logreg_submission2.head()

,SK_ID_CURR,TARGET
0,100001,0.076801
1,100005,0.081547
2,100013,0.040890
3,100028,0.024820
4,100038,0.066380


In [92]:
logreg_submission2.to_csv('submission2.csv', index=False)